In [ ]:
!pip install langchain openai
#!pip install langchain requests


In [1]:
# Укажите URL вашего сервера с моделью llama3-8b-instruct-8k
api_base = input("Введите адрес модели: ")
my_model = input("Введите название модели: ")
API_KEY = input("Введите ключ: ")

In [1]:
import os

# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Строим путь на три уровня вверх
config_path = os.path.join(current_dir, '..', '..', 'llama.config')

# Нормализуем путь (убираем лишние '..' и т.д.)
config_path = os.path.normpath(config_path)

# Читаем файл
with open(config_path, 'r') as file:
    api_base = file.readline().strip()
    my_model = file.readline().strip()
    API_KEY = file.readline().strip()

# Выводим значения
#print(f"API Base: {api_base}")
#print(f"My Model: {my_model}")
#print(f"API Key: {API_KEY}")


In [2]:
import os
#import logging
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage, HumanMessage

In [3]:

# Настройка ChatOpenAI
#llm = ChatOpenAI(
my_chat = ChatOpenAI(
    base_url = api_base,
    model = my_model,
    openai_api_key=API_KEY,  # Укажите ваш API-ключ (если требуется)
    temperature=0.7,                # Настройте параметры модели
    max_tokens=150
)

# Пример запроса
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

# Отправка запроса
response = my_chat(messages)

# Вывод ответа
print(response.content)


C:\Users\user\AppData\Local\Temp\ipykernel_28936\161643322.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  my_chat = ChatOpenAI(
C:\Users\user\AppData\Local\Temp\ipykernel_28936\161643322.py:18: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = my_chat(messages)


The capital of France is Paris.


### **7. Пример работы в Jupyter Notebook**

```
Чат с Llama 3 (Для выхода введите 'стоп')
You: расскажи что-нибуть интересное
Llama 3: Конечно! Вот интересный факт: в 1952 году американский физик и изобретатель Роберт Вуд провел эксперимент, который стал классическим примером оптической иллюзии. Он построил устройство, которое позволяло ему исчезнуть на глазах у зрителей. Вуд использовал зеркала и специальное освещение, чтобы создать впечатление, что он исчезает и появляется в другом месте. Этот эксперимент стал популярным в научных шоу и показал, как оптические иллюзии могут обман
You: кто такой роберт вуд?
Llama 3: Роберт Вуд (Robert W. Wood) — это имя нескольких известных личностей, но наиболее известен из них физик и изобретатель, живший в конце XIX — начале XX века. Однако, если вы говорите о более современной фигуре, возможно, вы имеете в виду компанию или персону с таким же именем.

1. **Роберт Вуд (1868-1955)** — американский физик и изобретатель, известный своими экспериментами с оптикой и инфракрасным излучением. Он также написал несколько научно-популярных книг
Чат завершен
```


In [4]:
# Настройка ChatOpenAI для LLMChain

llm = ChatOpenAI(
    base_url=api_base,
    model=my_model,
    openai_api_key=API_KEY,  # Укажите ваш API-ключ (если требуется)
    temperature=0.7,          # Настройте параметры модели
    max_tokens=150
)

# Создаем шаблон для чата
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant"),
    MessagesPlaceholder(variable_name="input"),  # Используем MessagesPlaceholder
])

# Создаем цепочку (chain)
chain = LLMChain(llm=llm, prompt=prompt)

def chat():
    print("Чат с Llama 3 (Для выхода введите 'стоп')")
    while True:
        my_input = input("Вы: ")
        if my_input.lower() == "стоп":
            print("Чат завершен")
            break

        try:
            # Передаем input в chain.run() как словарь
            response = chain.run(input=[HumanMessage(content=my_input)])
            print(f"You: {my_input}")

            print(f"Llama 3: {response}")
        except Exception as e:
            print("Произошла ошибка. Пожалуйста, попробуйте снова!")


C:\Users\user\AppData\Local\Temp\ipykernel_28936\4013588632.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [5]:
chat()

Чат с Llama 3 (Для выхода введите 'стоп')
Чат завершен


Подключил Structured Outputs отсюда:

https://docs.vllm.ai/en/latest/features/structured_outputs.html

In [6]:
from openai import OpenAI
client = OpenAI(
    base_url=api_base,
    api_key=API_KEY,
)

In [7]:
completion = client.chat.completions.create(
    model=my_model,
    messages=[
        {"role": "user", "content": "Classify this sentiment: The world is bautiful!"}
    ],
    extra_body={"guided_choice": ["positive", "negative"]},
)
print(completion.choices[0].message.content)

positive


In [8]:
completion = client.chat.completions.create(
    model=my_model,
    messages=[
        {"role": "user", "content": "Classify this sentiment: I see trees of green, red roses too. I see them bloom for me and you. And I think to myself: What a wonderful world!"}
    ],
    extra_body={"guided_choice": ["positive", "negative"]},
)
print(completion.choices[0].message.content)

positive


In [9]:
completion = client.chat.completions.create(
    model=my_model,
    messages=[
        {"role": "user", "content": "Classify this sentiment: They are ugly and terrifying!"}
    ],
    extra_body={"guided_choice": ["positive", "negative"]},
)
print(completion.choices[0].message.content)

negative
